## Métodos de potencia inversa

En esta sección nos enfocaremos al problema numérico de encontrar el eigenvalor mas pequeño $\lambda_1$ para la que existan soluciones no triviales de  $\mathbf{Ax} = \lambda\mathbf{x}$.

Si los eigenvectores $x_i$ de $A$ son ortonormales, se pueden expresar en una base.

\begin{equation*}
v = \sum_{i=1}^{n} v_i x_i ~~~~ z = \sum_{i=1}^{n} z_i x_i
\end{equation*}
	
y sustituyendo en $Az=v$ 
	
\begin{equation*}
\sum_{i=1}^{n}(z_i \lambda _i-v_i)x_i=0~entonces~z_i=\frac{v_i}{\lambda_i}
\end{equation*}
	
entonces

\begin{equation*}
z_i = \frac{1}{\lambda_1}\left(v_1 x_1 + v_2 \frac{\lambda_1}{\lambda_2} x_2 + v_3 \frac{\lambda_1}{\lambda_3} x_3 + \cdots \right)
\end{equation*}

1. Si $\lambda_1 / \lambda_i < 1$, $z$ es la mejor aproximación para $x_1$, entonces se completa la primera interación. 
2. En los subsecuentes ciclos $v=z/|z|$, se repite el proceso y se incrementa la dominancia del primer término, convergiendo a:


\begin{equation*}
z=\frac{1}{\lambda_1} v_1 x_1 = \frac{1}{\lambda_1} x_1 ~~~~(v_1=1,v_2=v_3=\cdots =0)
\end{equation*}

Nuestro punto de partida es:

1. $v$ vector unitario que puede ser aleatorio.
2. Resolver $Az=v$ para $z$.
3. Calcular $|z|$.
4. Calcular $v=z/|z|$.

Este procedimiento se sigue hasta que $v$ sea despreciable, finalmente tenemos que 

\begin{equation*}
|z|=\pm \frac{1}{\lambda_1},~~~~ v=x_1 
\end{equation*}

si $z$ cambia de signo en interaciones sucesivas, $\lambda_1<0$, si no es así, es positiva.

La convergencia se determina con la desigualdad $|\lambda_1/\lambda_2<1|$. Si $|\lambda_2|$ es mucho mayor que  $|\lambda_1|$ la convergencia es rápida, de lo contrario la convergencia será muy lenta.

Se puede aplicar un factor de corrimiento $s$ a los eigenvalores, tal que 

\begin{equation*}
\lambda=\lambda^{\ast}+s
\end{equation*}

Para transformar $\mathbf{Ax} = \lambda\mathbf{x}$:

\begin{equation*}
Ax=(\lambda^{\ast}+s)x, ~~~~ A^{\ast}=A-sI
\end{equation*}
 
Donde el eigenvalor del problema original es $\lambda = \lambda^{\ast}_1 + s$


Se intercambia $v$ por $z$ (eigenvalor mas grande):

1. $v$ vector unitario que puede ser aleatorio.
2. Resolver $z=vA$ para $z$.
3. Calcular $|z|$.
4. Calcular $v=z/|z|$.

Este procedimiento se sigue hasta que $v$ sea despreciable, finalmente tenemos que 

\begin{equation*}
|z|=\pm \lambda_n,~~~~ v=x_n 
\end{equation*}

si $z$ cambia de signo en interaciones sucesivas, $\lambda_1<0$, si no es así, es positiva.



# Ejemplo 1: Método de potencia

La matriz de tensión en un punto es:

\begin{align*}
S=\begin{bmatrix}
-30 & 10 & 20 \\
10 & 40 & -50  \\
20 & -50 & -10 
\end{bmatrix} MPa
\end{align*}

Calcular el valor mayor para la tensión por el método de potencia con el eigenvector inicial $v=[1~0~0]^T$

In [1]:
#Método de potencia
import numpy as np
import math

s = np.array([[-30.0,10.0,20.0],[10.0,40.0,-50.0],[20.0,-50.0,-10.0]])
v = np.array([1.0, 0.0, 0.0])
for i in range(100):
  vOld = v.copy()
  z = np.dot(s,v)
  zMag = math.sqrt(np.dot(z,z))
  v = z/zMag
  if np.dot(vOld,v) < 0.0:
    sign = -1.0
    v = -v
  else: sign = 1.0
  if math.sqrt(np.dot(vOld - v,vOld - v)) < 1.0e-6: break
lam = sign*zMag
print("# de interaciones =",i)
print("Eigenvalor =",lam)

# de interaciones = 92
Eigenvalor = 70.94348330679053


# Ejemplo 2: Método de potencia inversa

Calcular el eigenvalor mas pequeño $\lambda_1$ que corresponde al eigenvector

\begin{align*}
	A=\begin{bmatrix}
		11 & 2 & 3 & 1 & 4 \\
		2 & 9 & 3 & 5 & 2  \\
		3 & 3 & 15 & 4 & 3 \\
     	1 & 5 & 4 & 12 & 4 \\
		4 & 2 & 3 & 4 & 17 \\
	\end{bmatrix}
\end{align*}
	
por el método de potencia inversa con un corrimiento en el eigenvalor de $\lambda_1\approx5$


In [2]:
#Método de potencia inversa

## modulo LUdecomp                                                      
import numpy as np
import math
from random import random

def LUdecomp(a):
    n = len(a)
    for k in range(0,n-1):
        for i in range(k+1,n):
           if a[i,k] != 0.0:
               lam = a [i,k]/a[k,k]
               a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
               a[i,k] = lam
    return a

def LUsolve(a,b):
    n = len(a)
    for k in range(1,n):
        b[k] = b[k] - np.dot(a[k,0:k],b[0:k])
    b[n-1] = b[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
       b[k] = (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k]
    return b

def inversePower(a,s,tol=1.0e-6):
    n = len(a)
    aStar = a - np.identity(n)*s                
    aStar = LUdecomp(aStar)                            
    x = np.zeros(n)
    for i in range(n):                    
        x[i] = random()
    xMag = math.sqrt(np.dot(x,x))                        
    x =x/xMag
    for i in range(50):                              
        xOld = x.copy()                            
        x = LUsolve(aStar,x)                    
        xMag = math.sqrt(np.dot(x,x))                  
        x = x/xMag
        if np.dot(xOld,x) < 0.0:          
            sign = -1.0
            x = -x
        else: sign = 1.0
        if math.sqrt(np.dot(xOld - x,xOld - x)) < tol:
            return s + sign/xMag,x
    print('No converge')

In [3]:
s = 5.0
a = np.array([[11.0,2.0,3.0,1.0,4.0], \
              [2.0,9.0,3.0,5.0,2.0], \
              [3.0,3.0,15.0,4.0,3.0], \
              [1.0,5.0,4.0,12.0,4.0], \
              [4.0,2.0,3.0,4.0,17.0]])
lam,x = inversePower(a,s)
print("Eigenvalor =",lam)
print("\nEigenvector:\n",x)

Eigenvalor = 4.873946378649167

Eigenvector:
 [-0.26726605  0.74142853  0.05017273 -0.59491453  0.14970634]
